<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pronto transformers[torch] datasets evaluate \
&& pip install accelerate -U \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1

In [31]:
import random
import pronto
import torch
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, BioGptTokenizer, BioGptModel, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

In [3]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")

In [4]:
# Get subsumptions from CSV file to a dictionary

def get_mappings_from_file(filename):
  mappings = {}
  with open(filename) as f:
    for line in f:
      source_id, target_id = line.strip().split(',')
      mappings[source_id] = target_id
  return mappings

In [7]:
equiv_doid2ncit = get_mappings_from_file("equiv_doid2ncit.csv")
equiv_ncit2doid = get_mappings_from_file("equiv_ncit2doid.csv")
subs_doid2ncit = get_mappings_from_file("subs_doid2ncit.csv")
subs_ncit2doid = get_mappings_from_file("subs_ncit2doid.csv")
neg_subs_doid2ncit = get_mappings_from_file("neg_subs_doid2ncit.csv")
neg_subs_ncit2doid = get_mappings_from_file("neg_subs_ncit2doid.csv")

# Convert relations to sentences

> Currently considering parents, children & siblings for conceptual reasons, but could also take 'n-hop' appraoch, e.g. 1-hop only with parents and children, or 2-hop to include grandparents, grandchildren and siblings.

> How do I incorporate the desired mapping for training? Should I incorporate both all this AND target info, or too much? Could be SELF + desired relatives instead, or SELF + PARENT + DESIRED PARENT, etc.

In [60]:
entity_markers = ["[SUB]", "[/SUB]", "[SUP]", "[/SUP]"]
sep_token = "[SEP]"
cls_token = "[CLS]"

In [17]:
# Create sentence from the given entity, containing its direct parents & siblings

def get_sentence(entity_id, onto):
  sub_in, sub_out, sup_in, sup_out = entity_markers

  subsumer = onto.get_term(entity_id)
  supersumers = list(subsumer.superclasses(distance=1, with_self=False))

  sentence = [sub_in, subsumer.name, sub_out]
  for supersumer in supersumers:
    sentence.extend([sup_in, supersumer.name, sup_out])

  return "".join(sentence)

In [70]:
# Create sentence from the given source entity, containing its mapping's parent & siblings

def get_combined_sentence(source_id, target_id, source_onto, target_onto):
  sub_in, sub_out, sup_in, sup_out = entity_markers

  if source_id not in source_onto.terms() or target_id not in target_onto.terms():
    return

  subsumer = source_onto.get_term(source_id)
  equivalent = target_onto.get_term(target_id)
  parents = list(equivalent.superclasses(distance=1, with_self=False))

  if len(parents) != 1:
    return
  supersumer = parents[0]

  #sentence = "".join([sub_in, subsumer.name, sub_out, sup_in, supersumer.name, sup_out])
  sentence = (sep_token + cls_token).join([subsumer.name, supersumer.name])
  return sentence

In [71]:
source_id = "DOID:0014667"
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
# tokenizer.add_tokens(entity_markers)

sentence = get_combined_sentence(source_id, subs_doid2ncit[source_id], doid, ncit)
tokenized = tokenizer(sentence)

print(sentence)
tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

disease of metabolism[SEP][CLS]Non-Neoplastic Disorder


['[CLS]',
 'disease',
 'of',
 'metabolism',
 '[SEP]',
 '[CLS]',
 'non',
 '-',
 'neoplastic',
 'disorder',
 '[SEP]']

# Train end-to-end BERT model

In [63]:
def generate_labelled_samples(subs, negsubs, source_onto, target_onto):
  random.seed(3)
  samples = []
  labels = []

  pairs = list(subs.items()) + list(negsubs.items())
  zeros_and_ones = [0] * len(negsubs) + [1] * len(subs)
  labelled_pairs = list(zip(zeros_and_ones, pairs))
  random.shuffle(labelled_pairs)

  for label, (source_id, target_id) in tqdm(labelled_pairs):
    sample_sentence = get_combined_sentence(source_id, target_id, source_onto, target_onto)
    if sample_sentence:
      samples.append(sample_sentence)
      labels.append(label)

  print()
  return samples, labels

In [72]:
samples, labels = generate_labelled_samples(subs_doid2ncit, neg_subs_doid2ncit, doid, ncit)
print("Samples: ", len(samples))
print("Labels: ", len(labels))

100%|██████████| 3766/3766 [00:00<00:00, 7864.64it/s]


Samples:  2775
Labels:  2775


In [65]:
X = np.array(samples)
y = np.array(labels)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=3)

In [37]:
id2label = {ind:label for (ind, label) in enumerate(labels)}
label2id = {label:ind for (ind, label) in enumerate(labels)}

dataset_train = Dataset.from_dict({'sample':X_train, 'label':[label2id[label] for label in y_train]})
dataset_val = Dataset.from_dict({'sample':X_val, 'label':[label2id[label] for label in y_val]})
dataset_test = Dataset.from_dict({'sample':X_test, 'label':[label2id[label] for label in y_test]})
dataset = DatasetDict({'train':dataset_train,'val':dataset_val,'test':dataset_test})

In [38]:
def compute_metrics(eval_pred):
  accuracy = evaluate.load("accuracy")
  macro_metrics = {'precision' : evaluate.load("precision"),
                   'recall' : evaluate.load('recall'),
                   'f1' : evaluate.load('f1')}

  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  metric_dict = {name : metric.compute(predictions=predictions, references=labels, average='macro') for (name, metric) in macro_metrics.items()}
  metric_dict['accuracy'] = accuracy.compute(predictions=predictions, references=labels)
  return metric_dict

In [66]:
def train_model(dataset, id2label, label2id, entity_markers=entity_markers, learning_rate=1e-4, epochs=1, batch_size=16):
  pretrained = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

  tokenizer = AutoTokenizer.from_pretrained(pretrained)
  # tokenizer.add_tokens(entity_markers)
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})

  preprocess_tokenize = lambda examples: tokenizer(examples["sample"], padding="longest")
  tokenized_data = dataset.map(preprocess_tokenize, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
  model = AutoModelForSequenceClassification.from_pretrained(
      pretrained, num_labels=len(id2label), id2label=id2label, label2id=label2id
  )
  model.resize_token_embeddings(len(tokenizer))
  model.config.pad_token_id = tokenizer.pad_token_id

  training_args = TrainingArguments(
    output_dir="testing",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_data['train'],
      eval_dataset=tokenized_data['val'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  return trainer, tokenized_data

In [67]:
torch.cuda.empty_cache()
trainer, tokenized_data = train_model(dataset, id2label, label2id)
trainer.train()

Map:   0%|          | 0/1665 [00:00<?, ? examples/s]

Map:   0%|          | 0/555 [00:00<?, ? examples/s]

Map:   0%|          | 0/555 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.146201,{'precision': 0.9654359621790051},{'recall': 0.9757227575982209},{'f1': 0.9698778833107191},{'accuracy': 0.9711711711711711}


Trainer is attempting to log a value of "{'precision': 0.9654359621790051}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9757227575982209}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9698778833107191}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9711711711711711}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=105, training_loss=0.9815120152064732, metrics={'train_runtime': 24.333, 'train_samples_per_second': 68.426, 'train_steps_per_second': 4.315, 'total_flos': 49100586834852.0, 'train_loss': 0.9815120152064732, 'epoch': 1.0})

In [69]:
predictions, label_ids, metrics = trainer.predict(tokenized_data['val'])
labels_predicted = [id2label[prediction] for prediction in np.argmax(predictions, axis=1)]
for metric in metrics.items():
  print(metric)

('test_loss', 0.14620058238506317)
('test_precision', {'precision': 0.9654359621790051})
('test_recall', {'recall': 0.9757227575982209})
('test_f1', {'f1': 0.9698778833107191})
('test_accuracy', {'accuracy': 0.9711711711711711})
('test_runtime', 4.8586)
('test_samples_per_second', 114.231)
('test_steps_per_second', 7.204)
